In [88]:
import time
import networkx as nx
with open('input.txt') as f:
    lines = f.read().splitlines()

## Part 1

We need to through every node and add edges to all adjacent nodes if the pipes are connected
- F connecets to the node right and down
- | connects to the node up and down
- \- connects to the node left and right
- 7 connects to the node left and down
- J connects to the node left and up
- L connects to the node right and up
- S connects to all adjacent nodes that connect to it

In [89]:
starttime = time.perf_counter_ns()
pipes = []
for line in lines:
    pipes.append([x for x in line])
Maze = nx.Graph()
# add the nodes
for y in range(len(pipes)):
    for x in range(len(pipes[y])):
        Maze.add_nodes_from([(x,y)], value=pipes[y][x])
# create the edges with some beautiful compact code
for y in range(len(pipes)):
    for x in range(len(pipes[y])):
        if pipes[y][x] == 'F':
            if x+1 < len(pipes[y]):
                if pipes[y][x+1] == '7' or pipes[y][x+1] == '-' or pipes[y][x+1] == 'J' or pipes[y][x+1] == 'S':
                    Maze.add_edge((x,y),(x+1,y))
            if y+1 < len(pipes):
                if pipes[y+1][x] == 'J' or pipes[y+1][x] == '|' or pipes[y+1][x] == 'L' or pipes[y+1][x] == 'S':
                    Maze.add_edge((x,y),(x,y+1))
        elif pipes[y][x] == '|':
            if y+1 < len(pipes):
                if pipes[y+1][x] == 'J' or pipes[y+1][x] == '|' or pipes[y+1][x] == 'L' or pipes[y+1][x] == 'S':
                    Maze.add_edge((x,y),(x,y+1))
            if y-1 >= 0:
                if pipes[y-1][x] == 'F' or pipes[y-1][x] == '|' or pipes[y-1][x] == '7' or pipes[y-1][x] == 'S':
                    Maze.add_edge((x,y),(x,y-1))
        elif pipes[y][x] == '-':
            if x+1 < len(pipes[y]):
                if pipes[y][x+1] == '7' or pipes[y][x+1] == '-' or pipes[y][x+1] == 'J' or pipes[y][x+1] == 'S':
                    Maze.add_edge((x,y),(x+1,y))
            if x-1 >= 0:
                if pipes[y][x-1] == 'F' or pipes[y][x-1] == '-' or pipes[y][x-1] == 'L' or pipes[y][x-1] == 'S':
                    Maze.add_edge((x,y),(x-1,y))
        elif pipes[y][x] == '7':
            if x-1 >= 0:
                if pipes[y][x-1] == 'F' or pipes[y][x-1] == '-' or pipes[y][x-1] == 'L' or pipes[y][x-1] == 'S':
                    Maze.add_edge((x,y),(x-1,y))
            if y+1 < len(pipes):
                if pipes[y+1][x] == 'J' or pipes[y+1][x] == '|' or pipes[y+1][x] == 'L' or pipes[y+1][x] == 'S':
                    Maze.add_edge((x,y),(x,y+1))
        elif pipes[y][x] == 'J':
            if x-1 >= 0:
                if pipes[y][x-1] == 'F' or pipes[y][x-1] == '-' or pipes[y][x-1] == 'L' or pipes[y][x-1] == 'S':
                    Maze.add_edge((x,y),(x-1,y))
            if y-1 >= 0:
                if pipes[y-1][x] == 'F' or pipes[y-1][x] == '|' or pipes[y-1][x] == '7' or pipes[y-1][x] == 'S':
                    Maze.add_edge((x,y),(x,y-1))
        elif pipes[y][x] == 'L':
            if x+1 < len(pipes[y]):
                if pipes[y][x+1] == '7' or pipes[y][x+1] == '-' or pipes[y][x+1] == 'J' or pipes[y][x+1] == 'S':
                    Maze.add_edge((x,y),(x+1,y))
            if y-1 >= 0:
                if pipes[y-1][x] == 'F' or pipes[y-1][x] == '|' or pipes[y-1][x] == '7' or pipes[y-1][x] == 'S':
                    Maze.add_edge((x,y),(x,y-1))
        elif pipes[y][x] == 'S':
            if x+1 < len(pipes[y]):
                if pipes[y][x+1] == '7' or pipes[y][x+1] == '-' or pipes[y][x+1] == 'J' or pipes[y][x+1] == 'S':
                    Maze.add_edge((x,y),(x+1,y))
            if y+1 < len(pipes):
                if pipes[y+1][x] == 'J' or pipes[y+1][x] == '|' or pipes[y+1][x] == 'L' or pipes[y+1][x] == 'S':
                    Maze.add_edge((x,y),(x,y+1))
            if x-1 >= 0:
                if pipes[y][x-1] == 'F' or pipes[y][x-1] == '-' or pipes[y][x-1] == 'L' or pipes[y][x-1] == 'S':
                    Maze.add_edge((x,y),(x-1,y))
            if y-1 >= 0:
                if pipes[y-1][x] == 'F' or pipes[y-1][x] == '|' or pipes[y-1][x] == '7' or pipes[y-1][x] == 'S':
                    Maze.add_edge((x,y),(x,y-1))

# find the start node
start = next((node for node in Maze.nodes if Maze.nodes[node]['value'] == 'S'), None)

# now look for loops in the graph that contain start node
ciclo = next((cycle for cycle in nx.cycle_basis(Maze) if start in cycle), None)

endtime = time.perf_counter_ns()
print("We need", int(len(ciclo)/2), "steps to get to the point furthest away from the start.")
print("Time taken:", round((endtime-starttime)/1000000, 3), "ms")

We need 7107 steps to get to the point furthest away from the start.
Time taken: 155.248 ms


## Part 2

Consider moving diagonally and counting the number of walls from the cycle passed until hitting the border

In [90]:
import multiprocessing as mp

def count_inside(node):
    if node not in ciclo:
        hit, x, y = 0, node[0], node[1]
        while x < len(pipes[0]) and y < len(pipes):
            # if the pipe is in this position is part of the cycle, we hit a wall, unless it's a '7' or 'L'
            if (x, y) in ciclo_values and ciclo_values[(x, y)] != '7' and ciclo_values[(x, y)] != 'L':
                hit += 1
            x += 1
            y += 1
        if hit % 2 != 0:
            return 1
    return 0

starttime = time.perf_counter_ns()

# Dictionary for faster lookup (reduces time taken by ~10 times because lookup is O(1) instead of O(n))
ciclo_values = {(x, y): Maze.nodes[(x, y)]['value'] for x, y in ciclo}

# multiprocessing pool
pool = mp.Pool(mp.cpu_count())

# Calculate the number of points inside the maze in parallel
inside = sum(pool.map(count_inside, Maze.nodes))

pool.close()

print("There are", inside, "points inside the maze.")
print("Time taken:", round((time.perf_counter_ns()-starttime)/1000000, 3), "ms")

There are 281 points inside the maze.
Time taken: 932.691 ms


## Maze Visualization

In [91]:
# cool visualization of the maze using these ASCII characters:
# ┐ └ ┘ ┌ ─ │
for y in range(len(pipes)):
    for x in range(len(pipes[y])):
        if (x,y) in ciclo:
            if pipes[y][x] == 'F':
                print('┌', end='')
            elif pipes[y][x] == '|':
                print('│', end='')
            elif pipes[y][x] == '-':
                print('─', end='')
            elif pipes[y][x] == '7':
                print('┐', end='')
            elif pipes[y][x] == 'J':
                print('┘', end='')
            elif pipes[y][x] == 'L':
                print('└', end='')
            elif pipes[y][x] == 'S':
                print('S', end='')
        else:
            print('.', end='')
    print()

............................................................................................................................................
............................................................................................................................................
............................................................................................................................................
............................................................................................................................................
....................................................┌──┐┌─┐┌┐.....................................┌┐┌┐......................................
...................................................┌┘┌─┘│┌┘││┌┐........................┌┐........┌┘││└─┐....┌┐..............................
.................................................┌┐└┐│..│└┐│└┘│........................││........└┐└┘┌─┘┌┐┌┐│└┐┌┐...........................
.............